# Test of completeness for large dataset (million or billions)

Implement a random test case on millions or billions of data rows using PySpark


In [ ]:
!pip install pyspark
!pip install findspark
!pip install unidecode

In [4]:
import pandas as pd
import pyspark as spark

import findspark
findspark.init()
     

# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
print(spark)
     

from pyspark import SparkContext
from pyspark.sql import SQLContext 

In [2]:
!curl https://raw.githubusercontent.com/vamsikrishnaprasad/predictive-Analytics-for-Retail-Banking/master/bank.csv --output bank.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  908k  100  908k    0     0  1678k      0 --:--:-- --:--:-- --:--:-- 1675k


In [5]:
bank_df = spark.read.option("header",True).csv("bank.csv")

In [6]:
bank_df.show(5, False)

+---+----------+-------+---------+-----+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job       |marital|education|fault|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-----+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|59 |admin.    |married|secondary|no   |2343   |yes    |no  |unknown|5  |may  |1042    |1       |-1   |0       |unknown |yes    |
|56 |admin.    |married|secondary|no   |45     |no     |no  |unknown|5  |may  |1467    |1       |-1   |0       |unknown |yes    |
|41 |technician|married|secondary|no   |1270   |yes    |no  |unknown|5  |may  |1389    |1       |-1   |0       |unknown |yes    |
|55 |services  |married|secondary|no   |2476   |yes    |no  |unknown|5  |may  |579     |1       |-1   |0       |unknown |yes    |
|54 |admin.    |married|tertiary |no   |184    |no     |no  |unknown|5  |may  |673     |2 

In [9]:
FRACTION = 0.1 # sampling 10% of rows
bank_df.sample(fraction = FRACTION).show(5, False)

bank_df.sample(fraction = FRACTION, seed=100).show(5, False)
bank_df.sample(fraction = FRACTION, seed=100).show(5, False)
bank_df.sample(fraction = FRACTION, seed=200).show(5, False)

+---+-----------+--------+---------+-----+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job        |marital |education|fault|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-----+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|56 |management |married |tertiary |no   |830    |yes    |yes |unknown|6  |may  |1201    |1       |-1   |0       |unknown |yes    |
|41 |admin.     |married |secondary|no   |55     |yes    |no  |unknown|8  |may  |1120    |2       |-1   |0       |unknown |yes    |
|28 |admin.     |divorced|secondary|no   |785    |yes    |no  |unknown|8  |may  |442     |2       |-1   |0       |unknown |yes    |
|26 |blue-collar|single  |secondary|no   |82     |yes    |no  |unknown|9  |may  |654     |1       |-1   |0       |unknown |yes    |
|42 |technician |single  |secondary|no   |1364   |yes    |no  |unknown|15 |m

This particular use case was rather interesting and allowed me to put into practice the sample() function. I had a number of CSV file on the one hand, notably from a different source system, and had to check that the data existed in the actual dataset, containing billions of rows. As it wasn’t feasible to validate every single row, I generated some sample data from the CSV file, placed that data in a pandas dataframe to later convert it into a list, then using the filter() command, extracted the row from the actual dataset, based on the primary keys. If results were returned from the filter() command, then I knew that the row existed in the actual dataset. If not — ERROR!!!

In [10]:
bank_df.count()

11162

In [18]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
StructField("id",IntegerType(),True),
StructField("name",StringType(),True),
StructField("age",IntegerType(),True)])

# This data is what we will sample
expected_data=[(1, "Brooke", 20),
(2, "Jon", 45),
(3, "Susan", 53),
(9, "Axl", 21)] # Axl is the data row that is NOT found in actual dataset

actual_data=[(1, "Brooke", 20),
(2, "Jon", 45),
(3, "Susan", 53),
(4, "Mary", 46),
(5, "Adam", 6),
(6, "Brian", 9),
(7, "Melanie", 5),
(8, "Sarah", 10)]

expected_df = spark.createDataFrame(data=expected_data, schema=schema)
actual_df = spark.createDataFrame(data=actual_data,schema=schema)

sample_df = expected_df.sample(fraction=0.6) # get a sample of the expected dataset

pandas_df = sample_df.toPandas()
sample_df.show() # sample values that will be searched

vals = pandas_df.values
listValues = vals.tolist() # turn pandas into a list, as it is a small number of results

print(listValues)

# Loop over the list of sampled data, checking whether it exists in the actual dataset
for line in listValues:
   count = 0
   primary_key = line[0]
   count = actual_df.filter(condition=actual_df.id==primary_key).count()
   if count == 0: print("Row not found " + str(primary_key))

+---+-----+---+
| id| name|age|
+---+-----+---+
|  2|  Jon| 45|
|  3|Susan| 53|
|  9|  Axl| 21|
+---+-----+---+

[[2, 'Jon', 45], [3, 'Susan', 53], [9, 'Axl', 21]]
Row not found 9
